COMENZAMOS INGESTANDO LOS ARCHIVOS

In [2]:
import pandas as pd

dfetl= pd.read_csv('C:/Users/acedo/OneDrive/Escritorio/Pi 1/FastApi/ETL.csv')


In [3]:
df1 = pd.read_csv('C:/Users/acedo/OneDrive/Escritorio/Pi 1/datasets_rating/ratings/1.csv')
df2 = pd.read_csv('C:/Users/acedo/OneDrive/Escritorio/Pi 1/datasets_rating/ratings/2.csv')
df3 = pd.read_csv('C:/Users/acedo/OneDrive/Escritorio/Pi 1/datasets_rating/ratings/3.csv')
df4 = pd.read_csv('C:/Users/acedo/OneDrive/Escritorio/Pi 1/datasets_rating/ratings/4.csv')
df5 = pd.read_csv('C:/Users/acedo/OneDrive/Escritorio/Pi 1/datasets_rating/ratings/5.csv')
df6 = pd.read_csv('C:/Users/acedo/OneDrive/Escritorio/Pi 1/datasets_rating/ratings/6.csv')
df7 = pd.read_csv('C:/Users/acedo/OneDrive/Escritorio/Pi 1/datasets_rating/ratings/7.csv')
df8 = pd.read_csv('C:/Users/acedo/OneDrive/Escritorio/Pi 1/datasets_rating/ratings/8.csv')



In [4]:
frames =[df1,df2,df3,df4,df5,df6,df7,df8]
df_usuarios= pd.concat(frames, axis=0)

In [5]:
df_usuarios.head(1)

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680


REALIZO PEQUEÑOS AJUSTES A LAS TABLAS

In [6]:
#borro la columna timertamp
df_usuarios = df_usuarios.drop(['timestamp'], axis=1)

In [7]:
#borro columnas que no son de interes.
dfetl = dfetl.drop(['cast','country','show_id','director','duration','date_added','release_year','rating','description','duration_int','duration_type'], axis=1)

In [8]:
dfetl.head(1)

,type,title,listed_in,plataform,ID,score
0,movie,the grand seduction,"comedy, drama",amazon,as1,3.549127


In [9]:
#renombro la columna Id para luego poder unir los dos datafrem
dfetl = dfetl.rename(columns={'ID':'movieId'})

In [10]:
df_unido= pd.merge(left=df_usuarios,right=dfetl)

In [11]:
df_unido.head(1)

,userId,rating,movieId,type,title,listed_in,plataform,score
0,1,1.0,as680,tv show,the english civil war,"documentary, special interest",amazon,3.572383


In [12]:
#reordeno las columnas para trabajar mas comodo
column = df_unido.columns.tolist()
column = ['userId', 'movieId', 'type', 'title', 'listed_in', 'plataform', 'rating']
df_unido = df_unido[column]
df_unido.head(2)

,userId,movieId,type,title,listed_in,plataform,rating
0,1,as680,tv show,the english civil war,"documentary, special interest",amazon,1.0
1,583,as680,tv show,the english civil war,"documentary, special interest",amazon,4.5


In [13]:
#chequeamos los valores maximos y minimos de las valoraciones.
print(df_unido['rating'].min())
print(df_unido['rating'].max())

0.5
5.0


COMENZAMOS A PREPRARAR NUESTROS DF PARA ENTRAR AL MODELO

In [14]:
preparacion= df_unido.loc[:,["userId","movieId","rating"]]

In [15]:
#utilizo esta funcion para poder acortar un poco el df y que no pierda tanto tiempo en correr
df_simplificado = preparacion.sample(n=1000000, replace=True)

MACHINE LEARNIG
-MODELO DE RECOMENDACION
-

Este modelo intentara recomendar una pelicula a un cliente que valoro una pelicula, en funcion a las valoraciones de otros cliente sobre otras peliculas.

In [16]:
from sklearn.preprocessing import LabelEncoder

CODIFICACION

In [17]:
#codificamos la columna 'userId' para que la libreria surprise que va a ser el encargado de entrenar el modelo
codif= df_simplificado.movieId
# creamos objeto labelencoder y ajustamos la columna
labelE = LabelEncoder()
labelE.fit(codif)
codig_transformada = labelE.transform(codif)
#lo reemplazamos a show id original
df_simplificado.movieId = codig_transformada
df_simplificado.head()

,userId,movieId,rating
3361247,115381,6757,2.5
4302100,42800,10703,4.0
460724,1057,4874,3.5
2410972,9006,14854,1.5
262519,72480,20347,4.0


In [18]:
from surprise import Dataset, Reader

# convertimos los dataframe a dataset, esto permitira que el entrenamiento del modelo pueda llevarse de una manera mas sencilla
reader = Reader(line_format='user item rating', rating_scale=(0.5, 5))
data = Dataset.load_from_df(df_simplificado, reader)

ENTRENAMOS EL MODELO

In [19]:
from surprise import SVD

#Ccontruimos la data que vamos a entrenarla
data_train_surp = data.build_full_trainset()

# definimos el modelo
svd = SVD()

# entrenamos el modelo
svd.fit(data_train_surp)

In [20]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [21]:
predictions = svd.test(testset)

In [22]:
predictions

[Prediction(uid=70766, iid=16303, r_ui=3.5, est=3.864360100774948, details={'was_impossible': False}),
 Prediction(uid=733, iid=16700, r_ui=2.0, est=3.3667816960645918, details={'was_impossible': False}),
 Prediction(uid=24755, iid=8907, r_ui=5.0, est=4.496645715691281, details={'was_impossible': False}),
 Prediction(uid=66526, iid=13566, r_ui=4.0, est=3.5389101110094874, details={'was_impossible': False}),
 Prediction(uid=44481, iid=18502, r_ui=3.5, est=2.7490843486820062, details={'was_impossible': False}),
 Prediction(uid=25442, iid=21876, r_ui=5.0, est=3.58719302343325, details={'was_impossible': False}),
 Prediction(uid=1145, iid=16365, r_ui=3.5, est=3.550442323886246, details={'was_impossible': False}),
 Prediction(uid=60173, iid=859, r_ui=3.0, est=3.3775090241001244, details={'was_impossible': False}),
 Prediction(uid=117252, iid=19052, r_ui=2.5, est=3.4259682729072845, details={'was_impossible': False}),
 Prediction(uid=10002, iid=7231, r_ui=2.0, est=2.6502477345885085, details

In [30]:
svd.predict(21,1568,4)

Prediction(uid=21, iid=1568, r_ui=4, est=4.08917083150978, details={'was_impossible': False})